In [1]:
!pip install fairlearn dalex imbalanced-learn

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.1/234.1 KB 1.3 MB/s eta 0:00:00 MB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 4.9 MB/s eta 0:00:005.0 MB/s eta 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.3/258.3 KB 4.9 MB/s eta 0:00:00m eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 6.3 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 KB 5.3 MB/s eta 0:00:00
  Created wheel for dalex: filename=dalex-1.7.1-py3-none-any.whl size=1042815 sha256=75b719e34a8f7f9497785fda447fd0cee4b2935b3058297138e96bfb81c91cad
  Stored in directory: /home/abrahao/.cache/pip/wheels/fe/c2/41/63e006b3312a4e17299bed5f83d985dea872368f9ab16bb20f
Successfully built dalex


In [14]:
import pandas as pd  # Manipulação de dados
import numpy as np  # Operações matemáticas e arrays
import dalex as dx  # Explicabilidade de modelos
import seaborn as sns  # Visualização de dados, baseado no matplotlib
import matplotlib.pyplot as plt  # Visualização de dados

# Métricas de justiça
from fairlearn.metrics import demographic_parity_difference, equalized_odds_difference, MetricFrame

# Ferramentas de modelagem e avaliação
from sklearn.model_selection import train_test_split  # Divisão de dados
from sklearn.tree import DecisionTreeClassifier  # Classificador árvore de decisão
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_curve, auc  # Métricas de avaliação
from sklearn.utils import compute_class_weight, compute_sample_weight  # Cálculo de pesos

# Técnicas de balanceamento de dados
from imblearn.combine import SMOTEENN  # Combinação de Oversampling e Undersampling
from imblearn.under_sampling import RandomUnderSampler  # Undersampling
from imblearn.over_sampling import RandomOverSampler  # Oversampling

# Utilitários
from collections import Counter  # Contagem de elementos


In [17]:
# 1. Carregar os dados
data = pd.read_csv('dados_aleatorios_ajustados_45.csv')

In [18]:
# 2. Criar a variável alvo 'Evasao'
data['Evasao'] = data['Motivo da Evasão'].apply(lambda x: 1 if x != '0' else 0)

# 3. Selecionar as características (X) e a variável alvo (y)
features = ['Idade', 'Gênero', 'Série', 'Média de Notas', 'Frequência', 'Participação em Atividades Extracurriculares', 'Renda Familiar', 'Localização']
X = data[features]
y = data['Evasao']

# 4. Converter a coluna 'Frequência' para float, lidando com valores não string
X['Frequência'] = X['Frequência'].apply(lambda x: str(x).replace('%', '')).astype(float) / 100

# 5. Converter variáveis categóricas em numéricas
X = pd.get_dummies(X, drop_first=True)

# 6. Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 7. Treinar o modelo de árvore de decisão
model = DecisionTreeClassifier(random_state=42)
model.fit(X_train, y_train)

# 8. Fazer previsões
y_pred = model.predict(X_test)

# 10. Criar objeto DALEX para interpretação
explainer = dx.Explainer(model, X_train, y_train, label="Árvore de Decisão")

# 11. Análise de importância das variáveis
importance = explainer.model_parts()
importance.plot()

# 12. Análise de explicações individuais
individual_explanation = explainer.predict_parts(X_test.iloc[[0]], type='shap')
individual_explanation.plot()

# 9. Avaliar o modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Acurácia do modelo: {accuracy:.2f}')

# 13. Visualizar a matriz de confusão
confusion = confusion_matrix(y_test, y_pred)
print("\nMatriz de confusão")
print(confusion)
#plt.figure(figsize=(8, 6))
#sns.heatmap(confusion, annot=True, fmt='d', cmap='Blues', xticklabels=['Não Evadiu', 'Evadiu'], yticklabels=['Não Evadiu', 'Evadiu'])
#plt.ylabel('Real')
#plt.xlabel('Previsto')
#plt.title('Matriz de Confusão')
#plt.show()

# 14. Análise de viés preditivo
# Calcular as métricas de viés
sensitive_features = X_test[[col for col in X_test.columns if 'Gênero' in col]]
metric_frame = MetricFrame(metrics={"accuracy": accuracy_score}, y_true=y_test, y_pred=y_pred, sensitive_features=sensitive_features)

# Exibir a análise de viés
print(metric_frame.by_group)

# Calcular e imprimir diferenças de equidade
demographic_parity = demographic_parity_difference(y_true=y_test, y_pred=y_pred, sensitive_features=sensitive_features)
equalized_odds = equalized_odds_difference(y_true=y_test, y_pred=y_pred, sensitive_features=sensitive_features)
print(f'Diferença de Paridade Demográfica: {demographic_parity:.3f}')
print(f'Diferença de Probabilidades Igualadas: {equalized_odds:.3f}')


Preparation of a new explainer is initiated

  -> data              : 24000 rows 8 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 24000 values
  -> model_class       : sklearn.tree._classes.DecisionTreeClassifier (default)
  -> label             : Árvore de Decisão
  -> predict function  : <function yhat_proba_default at 0x7a8f5c1bf490> will be used (default)
  -> predict function  : Accepts pandas.DataFrame and numpy.ndarray.
  -> predicted values  : min = 0.0, mean = 0.36, max = 1.0
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = 0.0, mean = 0.0, max = 0.0
  -> model_info        : package sklearn

A new explainer has been created!


Acurácia do modelo: 0.55

Matriz de confusão
[[2471 1373]
 [1340  816]]
                  accuracy
Gênero_Masculino          
False             0.486468
True              0.608913
Diferença de Paridade Demográfica: 0.153
Diferença de Probabilidades Igualadas: 0.174
